In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from sklearn.model_selection import train_test_split
from tqdm import tqdm

np.random.seed(2022)

In [2]:
def eliq(T):
    a_liq = np.float32(np.array([-0.976195544e-15,-0.952447341e-13,\
                                 0.640689451e-10,\
                      0.206739458e-7,0.302950461e-5,0.264847430e-3,\
                      0.142986287e-1,0.443987641,6.11239921]));
    c_liq = np.float32(-80.0)
    T0 = np.float32(273.16)
    return np.float32(100.0)*np.polyval(a_liq,np.maximum(c_liq,T-T0))

def eice(T):
    a_ice = np.float32(np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,\
                      0.602588177e-7,0.615021634e-5,0.420895665e-3,\
                      0.188439774e-1,0.503160820,6.11147274]));
    c_ice = np.float32(np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07]))
    T0 = np.float32(273.16)
    return np.where(T>c_ice[0],eliq(T),\
                   np.where(T<=c_ice[1],np.float32(100.0)*(c_ice[3]+np.maximum(c_ice[2],T-T0)*\
                   (c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5])),\
                           np.float32(100.0)*np.polyval(a_ice,T-T0)))

def esat(T):
    T0 = np.float32(273.16)
    T00 = np.float32(253.16)
    omtmp = (T-T00)/(T0-T00)
    omega = np.maximum(np.float32(0.0),np.minimum(np.float32(1.0),omtmp))
    return np.where(T>T0,eliq(T),np.where(T<T00,eice(T),(omega*eliq(T)+(1-omega)*eice(T))))

def RH(T,qv,P0,PS,hyam,hybm):
    R = np.float32(287.0)
    Rv = np.float32(461.0)
    p = P0 * hyam + PS[:, None] * hybm # Total pressure (Pa)
    
    T = np.float32(T)
    qv = np.float32(qv)
    p = np.float32(p)
    
    return Rv*p*qv/(R*esat(T))

In [3]:
datasets = !ls
datasets = [x for x in datasets if "h1" in x]
spData = xr.open_mfdataset(datasets)

In [4]:
nntbp = spData["NNTBP"]
nnqbp = spData["NNQBP"]
p0 = spData["P0"]
ps = spData["NNPS"]
hyam = spData["hyam"]
hybm = spData["hybm"]
relhum = spData["RELHUM"]
tphystnd = spData["TPHYSTND"]
phq = spData["PHQ"]
newhum = xr.zeros_like(spData["NNQBP"])
lats = spData["lat"]
lons = spData["lon"]
for i in tqdm(range(len(lats))):
    for j in range(len(lons)):
        latIndex = i
        lonIndex = j
        R = 287
        Rv = 461
        p = ps[:, latIndex, lonIndex].expand_dims({"lev":30}, axis = 1)*hybm
        T = nntbp[:, :, latIndex, lonIndex]
        qv = nnqbp[:, :, latIndex, lonIndex]
        #newhum[:,:, latIndex, lonIndex] = Rv*p*qv/(R*esat(T))
        

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:10<00:00,  5.85it/s]


In [14]:
#esat
T0 = 273.16
T00 = 253.16
Tdiff = T0-T00
omtmp = ((T-T00)/Tdiff).astype("float32")
omega = max(0,min(1,omtmp.min().values))

In [31]:
#eliq
a_liq = np.float32(np.array([-0.976195544e-15,-0.952447341e-13,\
                      0.640689451e-10,\
                      0.206739458e-7,0.302950461e-5,0.264847430e-3,\
                      0.142986287e-1,0.443987641,6.11239921]))
c_liq = -80
T0 = 273.16
100*np.polyval(a_liq,max(c_liq,(T-T0).max().values))

5.8251553

In [32]:
#eice
a_ice = np.float32(np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,\
                  0.602588177e-7,0.615021634e-5,0.420895665e-3,\
                  0.188439774e-1,0.503160820,6.11147274]))
c_ice = np.float32(np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07]))
T0 = np.float32(273.16)

# return np.where(T>c_ice[0],eliq(T),\
#                np.where(T<=c_ice[1],np.float32(100.0)*(c_ice[3]+np.maximum(c_ice[2],T-T0)*\
#                (c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5])),\
#                        np.float32(100.0)*np.polyval(a_ice,T-T0)))

In [ ]:
return np.where(T>c_ice[0],eliq(T),\
               np.where(T<=c_ice[1],np.float32(100.0)*(c_ice[3]+np.maximum(c_ice[2],T-T0)*\
               (c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5])),\
                       np.float32(100.0)*np.polyval(a_ice,T-T0)))

In [39]:
np.polyval(a_ice, T-T0)

array([[4.2924881e-03, 1.4004707e-03, 1.0194778e-02, ..., 4.8176107e+00,
        5.3691878e+00, 5.7870641e+00],
       [4.3873787e-03, 1.4271736e-03, 1.0322094e-02, ..., 4.8212719e+00,
        5.3696518e+00, 5.7752981e+00],
       [4.3969154e-03, 1.4929771e-03, 1.0321140e-02, ..., 4.8248000e+00,
        5.3670998e+00, 5.7662473e+00],
       ...,
       [6.6871643e-03, 1.7199516e-03, 1.5058517e-02, ..., 4.4915743e+00,
        4.9263649e+00, 5.2395043e+00],
       [6.6838264e-03, 1.6565323e-03, 1.5062809e-02, ..., 4.4948816e+00,
        4.9274502e+00, 5.2321410e+00],
       [6.6151619e-03, 1.7199516e-03, 1.5041351e-02, ..., 4.4984579e+00,
        4.9334488e+00, 5.2278371e+00]], dtype=float32)

In [35]:
T-T0

<xarray.DataArray 'NNTBP' (time: 96, lev: 30)>
dask.array<sub, shape=(96, 30), dtype=float32, chunksize=(48, 30), chunktype=numpy.ndarray>
Coordinates:
    lat      float64 87.86
    lon      float64 357.2
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * time     (time) object 0000-11-04 00:00:00 ... 0000-11-05 23:30:00

In [12]:
min(1,omtmp.min().values)

array(-4.458523, dtype=float32)

In [103]:
omtmp.astype("float32")

<xarray.DataArray 'NNTBP' (time: 96, lev: 30)>
dask.array<truediv, shape=(96, 30), dtype=float32, chunksize=(48, 30), chunktype=numpy.ndarray>
Coordinates:
    lat      float64 87.86
    lon      float64 357.2
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * time     (time) object 0000-11-04 00:00:00 ... 0000-11-05 23:30:00

In [21]:
np.polyval([3,0,1], 5)

76

In [23]:
np.polyval([3,0,1], 5)

76